In [3]:
import torch
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']
plt.style.use('fivethirtyeight')

In [4]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return (Y)

x = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
k = torch.tensor([[0, 1], [2, 3]])

print (x, x.shape)
print (k, k.shape)

print (corr2d(x, k))

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]) torch.Size([3, 3])
tensor([[0, 1],
        [2, 3]]) torch.Size([2, 2])
tensor([[19., 25.],
        [37., 43.]])


In [5]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return (Y)

class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return (corr2d(x, self.weight) + self.bias)


In [6]:
X = torch.ones((6, 8))
X[:,  2:6] = 0
print (X, X.shape)

# K representing finite difference operator: edge detection
K = torch.tensor([[1.0, -1.0]])
print (K, K.shape)

Y = corr2d(X, K)

print (Y, Y.shape)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]]) torch.Size([6, 8])
tensor([[ 1., -1.]]) torch.Size([1, 2])
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]) torch.Size([6, 7])


In [7]:
corr2d(X.T, K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [44]:
conv2D = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))

print('Original X:')
print(X, X.shape)

print('Original Y to fit:')
Y = Y.reshape([1, 1, 6, 7])
print(Y, Y.shape)

print('Example of Conv2D:')
Z = conv2D(X)
print(Z, Z.shape)


Original X:
tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.]]]]) torch.Size([1, 1, 6, 8])
Original Y to fit:
tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]]) torch.Size([1, 1, 6, 7])
Example of Conv2D:
tensor([[[[ 0.0917, -0.3280,  0.0000,  0.0000,  0.0000,  0.4197,  0.0917],
          [ 0.0917, -0.3280,  0.0000,  0.0000,  0.0000,  0.4197,  0.0917],
          [ 0.0917, -0.3280,  0.0000,  0.0000,  0.0000,  0.4197,  0.0917],
          [ 0.0917, -0.3280,  0.0000,  0.0000,  0.0000,  0.4197,  0.0917],
          [ 0.0917, -0.3280,  0.0000,  0.0000,  

/Users/jsmidt/Library/Python/3.8/lib/python/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [45]:
lr = 0.03
for i in range(12):
    Y_hat = conv2D(X)
    l = ((Y_hat - Y)**2).sum()
    conv2D.zero_grad()
    l.backward()
    #print(conv2D.weight.grad)
    conv2D.weight.data[:] -= lr * conv2D.weight.grad
    if (i+1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.item():.3f}')

epoch 2, loss 9.358
epoch 4, loss 1.589
epoch 6, loss 0.275
epoch 8, loss 0.049
epoch 10, loss 0.010
epoch 12, loss 0.002


In [48]:
print (conv2D.weight.data.view(2))
print (Y)
print (Y_hat)

tensor([ 0.9967, -0.9904])
tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]])
tensor([[[[-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079]]]],
       grad_fn=<ConvolutionBackward0>)


In [75]:
X = torch.zeros((6,8))

X[1:6,0:8].fill_diagonal_(1.0)
X[0:6,0:8].fill_diagonal_(1.0)
X[2:6,0:8].fill_diagonal_(1.0)

X

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0.]])

In [78]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return (Y)


# K representing finite difference operator: edge detection
K = torch.tensor([[1.0, -1.0]])
print (K, K.shape)

print ("Normal X")
Y = corr2d(X, K)
print (Y)

print ("Transpose X")
Y = corr2d(X.T, K)
print (Y)

print ("Transpose K")
Y = corr2d(X, K.T)
print (Y)

tensor([[ 1., -1.]]) torch.Size([1, 2])
Normal X
tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  0.,  1.,  0.]])
Transpose X
tensor([[ 0.,  0.,  1.,  0.,  0.],
        [-1.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  1.],
        [ 0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  0.],
        [ 0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]])
Transpose K
tensor([[ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0., -1.,  0.,  0.]])
